In [ ]:
# IMPORT declarations

import os
import random
import re


# CLASS declarations

class Player:
    
    def __init__(self, name):
        self.name = name
        self.board = Board()
        self.hits_list = []
        self.create_ships()
        print_separator()
        print(f"Player {self.name} created! \nNow positionning the ships...")
        self.position_ships()

    def create_ships(self):
        self.carrier = Ship("Carrier", 5)
        self.battleship = Ship("Battleship", 4)
        self.cruiser = Ship("Cruiser", 3)
        self.submarine = Ship("Submarine", 3)
        self.destroyer = Ship("Destroyer", 3)
        self.ships = [self.carrier, self.battleship, self.cruiser, self.submarine, self.destroyer]
        
    def position_ships(self):
        for ship in self.ships:
            if self.position_ship(ship):
                self.add_ship(ship, ship.coords)
            display_board(self, self.board.board_input)
        return True
      
    def position_ship(self, ship):
        print_separator()
        print(f"Placing {self.name}'s {ship.model} of size {ship.size}!")
        if not debug_mode:
            while True:
                position_input = check_input("Please enter coords (0-9) (0-9) (right or bottom): ", "[0-9] [0-9] .+")
                if position_input:
                    break
        else:
            position_input = debug_pattern_position_ships[self.ships.index(ship)] # DEBUG
        coords_input = position_input.split(" ")
        i = int(coords_input[0])
        j = int(coords_input[1])
        d = coords_input[2]
        ship.update_coords((i,j), d)
        if check_position_ship(self, ship):
            self.board.update_filled_coords(ship.coords)
        else:
            self.position_ship(ship)
        return True
    
    def add_ship(self, ship, coords):
        for coord in coords:
            self.board.board_input[coord[0]][coord[1]] = "X"


class Ship:
    
    def __init__(self, model, size):
        self.model = model
        self.size = size
        self.life = size
        self.origin = []
        self.coords=[]
        
    def update_coords(self, origin, direction):
        self.coords=[]
        if direction == "bottom" or direction == "b":
            for i in range(self.size):
                # Going bottom: increment line, same column
                self.coords.append((list(origin)[0]+i, list(origin)[1])) 
        else:
            for i in range(self.size):
                # Going right: same line, increment column
                self.coords.append((list(origin)[0], list(origin)[1]+i)) 
        self.update_origin(self.coords)
    
    def update_origin(self, coords):
        self.origin = coords[0]
        

class Board:
    
    def __init__(self):
        self.board_input = [['-' for i in range(10)] for j in range(10)] # Board as player sees it
        self.board_output = [['-' for i in range(10)] for j in range(10)] # Board as opponent sees it
        self.filled_coords = [] # List of coords tuples with ship
                    
    def update_board_output(self, coords, hit):
        if hit:
            # Ship hit by strike: display X
            self.board_output[coords[0]][coords[1]] = "X"
        else:
            # Strike falls into sea: display O
            self.board_output[coords[0]][coords[1]] = "O"
                    
    def update_filled_coords(self, coords):
        self.filled_coords.extend([tuple(coord) for coord in coords])

        
# FUNCTIONS declarations

# Interface functions

def print_separator():
    print("\n##########################################\n")

def display_board(player, board_to_display):
    print("\n" + f"Board of {player.name}:" + "\n")
    print("\n".join([" ".join(board_to_display[i]) for i in range(10)]))
    
# Check functions

# Print message, takes input, and checks if input follows exactly regex_rule (fullmatch)
def check_input(message, regex_rule):
    input_user = input(message).strip()
    if re.fullmatch(regex_rule, input_user):
        return input_user
    else:
        print("Incorrect entry, please retry.")
        return False

# Check if position of ship fits on the board and does not overlap with another ship
def check_position_ship(player, ship):
    for coord in ship.coords:
        if tuple(coord) not in base_board:
            print("The ship would not fit on the board. Please retry.")
            return False
        if tuple(coord) in player.board.filled_coords:
            print("The ship would overlap with another ship. Please retry.")
            return False
    return True


def check_strike_input(attacker_input):
    coords_attacker_input = tuple(int(i) for i in attacker_input.split(" "))
    if coords_attacker_input in attacker.hits_list:
        print("You already hit this spot, please try again!")
        return False
    else:
        return coords_attacker_input
        
# Game functions

def start_game():
    select_first_attacker()
    start_strikes()
    
def select_first_attacker():
    global attacker, defender
    attacker = random.choice([p1, p2])
    if attacker == p1:
        defender = p2
    else:
        defender = p1
    print_separator()
    print(f"\nThe first attacker is {attacker.name}!")
    
def start_strikes():
    display_board(defender, defender.board.board_output)
    while True:
        strike_result = strike()
        if strike_result == False:
            break
    if flag_game_active:
        switch()
        
def strike():
    print_separator()
    if not debug_mode:
        while True:
            attacker_input = check_input("Please enter coords (0-9) (0-9) to strike: ", "[0-9] [0-9]")
            if attacker_input:
                break
    else: # DEBUG
        global debug_pattern_strikes
        attacker_input = debug_pattern_strikes[0]
        debug_pattern_strikes.remove(debug_pattern_strikes[0])
    coords_attacker_input = check_strike_input(attacker_input)
    attacker.hits_list.append(coords_attacker_input)
    return inflict_damages(coords_attacker_input) # Return FALSE if not damages inflicted
    
    
def inflict_damages(coords_attacker_input):
    if coords_attacker_input in defender.board.filled_coords:
        defender.board.update_board_output(coords_attacker_input, True)
        for ship in defender.ships:
            if coords_attacker_input in ship.coords:
                ship.life -= 1
                if ship.life == 0:
                        print(f"{ship.model} sunk!")
                        display_board(defender, defender.board.board_output)
                        defender.ships.remove(ship)
                        if defender.ships == []:
                            endgame()
                            return False
                else:
                    print(f"{ship.model} hit! Still {ship.life}/{ship.size} to go to sink it!")
                    display_board(defender, defender.board.board_output)
        return True
    else:
        print("Miss!")
        defender.board.update_board_output(coords_attacker_input, False)
        display_board(defender, defender.board.board_output)
        return False
    
def switch():
    global attacker, defender
    attacker, defender = defender, attacker
    print_separator()
    print(f"\nNow {attacker.name} attacks!")
    start_strikes()
    
def endgame():
    global flag_game_active
    print(f"\nYou sunk the last ship of {defender.name}! {attacker.name} wins!")
    flag_game_active = False


# GLOBAL VARIABLES declaration

base_board = [(i,j) for i in range(10) for j in range(10)]
flag_game_active = True


# GAME

print_separator()
print("----------------")
print("BATTLESHIPS GAME")
print("----------------")

if not debug_mode:
    p1 = Player(input("\nPlease enter your name: "))
    p2 = Player(input("\nPlease enter your name: "))
else: # DEBUG
    p1 = Player("Ting")
    p2 = Player("Eric")

start_game()


# DEBUG mode (computer plays automatically)

debug_mode = False # Set to True for computer to play all game automatically
debug_pattern_position_ships = ["0 0 r", "1 1 r", "2 2 r", "3 3 r", "4 4 r"]
debug_pattern_strikes = ["0 0", "0 1", "5 5","0 0", "0 1", "0 2", "0 3", "0 4", "1 1", "1 2", "1 3", "1 4", "2 2", "5 5", "0 2", "0 3", "0 4", "1 1", "1 2", "1 3", "1 4", "2 2", "2 3", "2 4", "3 3", "3 4", "3 5", "4 4", "4 5", "4 6"]


# SUGGESTED IMPROVEMENTS
#
# Change coordinates input system from '0 0 / 9 9' to 'A1 / J10'
# and create a function convert_coords that converts from A1 to '0 0' / J10 to '9 9'
# Show A...J and 1...10 when board is displayed
# Distinguish visually between hit ship (*) and sunk ship (X)
# Clarify / break down inflict_damages() function
# Comment, reorder & clean code